## Forward Mode AD


# Introduction

Automatic Differentiation allows for fast and precise derivitative computation. It is not, however, the only method for computing derivatives. 

Derivatives are a fundamental tool in mathematics with applications in a range of scientific fields.  Differentiation is core to many computational problems including finding extrema and zeroes of functions. Optimization problems such as gradient descent in neural networks depend on partial derivatives to incrementally move towards optimal solutions. Directional derivatives and the Jacobian matrix are utilized in a variety of contexts including backpropogation in training neural networks. Directional derivatives compute the change in a mult-dimensional function with a respect to any linear combination of its input parameters. 

Because of the prevalence of derivatives in computational fields, automatic differentiation focuses on developing algorithmic techniques for efficiently evaluating derivatives.


### Symbolic differentiation
Symbolic differentiation applies basic derivative rules to compute derivatives at the expression level. While precise, symbolic differentiation uses excess computational power and memory to hold the swelling equation resulting from chain rule. Symbolic differentiation computes the derivative of a function analytically, which offers high precision at the cost of execution time and exponential size in relation to the original function. Since many science applications care about the derivative evaluated at a value, we can often bypass the need for an analytic form for the derivative.

### Finite difference method
The finite difference method, on the other hand, estimates the derivative by computing the slope of the secant line, using a small change in $x$. The precision of the derivative depends on the choice of $x$; too large of a step leads to precision error but too small of a step creates rounding error.  Numerical differentiation, on the other hand, offers O(n) time computation for an n-dimensional gradient (think multi-dimensional derivative). It achieves this by estimating the derivative instead of computing it precisely. The trade-off, as with all estimations, is a reduction in accuracy due to rounding and truncation.

### Why AutoDiff?
Automatic Differentiation achieves the precision of symbolic differentiation with time complexity that matches the complexity of computing the original function. It does this by leveraging a useful fact: when computing a derivative at a specific point, we don't need the symbolic expression of the derivative. This allows us to compute our derivative via a series of computations that compute each depending on the prior computation rather than deriving the entire expression and plugging in the point at the end.

In forward mode, the graph can be abstract as computations are made on-the-fly and no storage of all prior computations is needed. In reverse-mode, on the other hand, space needs will exceed those of computing the original function as the computational graphs of derivatives will need to be stored and computed in reverse.

## Background

Atomatic differentiation (AD) is based on the property of decomposition of a function into intermediate steps based on the chain rule: $$\frac{dy}{dx} = \frac{dy}{du} \cdot \frac{du}{dx}$$
Consider the function $f(x_1, x_2) = e^{-(\sin(x_1) + \cos(x_2)}$. 

![](https://drive.google.com/uc?export=view&id=1q8eWiv6gv-ty2rv-S0OuEOqwmeVhsLgq)

The derivative is derived by chaining together the product of the outer derivative with each of the inner derivatives. This creates a computational graph, where each edge represents an elementary operation applied to an intermediary node.

Forward mode AD uses forward accumulation of these intermediary steps, traversing the chain rule in a partial ordering from the most inner part (the original inputs) working outward until the entire derivative has been evaluated. In addition, at each step we compute the derivative of the intermediate variables, called the tangent trace. This allows for us to compute the directional derivative in a single pass.

Dual numbers are convenient for differentiation since can allow us to capture both the primal and tangent trace at the same time. A dual number can be expressed as $a + b\epsilon$, where $a$ and $b$ are real numbers and $\epsilon$ is small enough such that $\epsilon^2 = 0$. The real part stores the primal trace while the dual part stores the tangent trace.

Forward mode automatic differentiation takes a seed vector which determines the direction of the derivative. This can be any linear combination of derivatives. To compute the full Jacobian using Forward Mode AD, this requires performing $m$ passes where $m$ is the number of inputs to the original function. On each pass, we calculate the derivative with respect to that direction.

In contrast, Reverse mode AD enables us to compute the full gradient in two passes. This requires storing an explicit computational graph. In the forward pass, reverse mode computes the primal trace and partial derivatives with respect to the parent node. In the reverse pass, these are reconstructed via the chain rule by accumulating the values of all child nodes.

Runtime of Forward and Reverse modes -

When a function is defined as $ \mathbb{R}^n \rightarrow \mathbb{R}^m $, Forward Mode computes the gradient with respect to the number of independent variables.
Therefore, the computational cost of using Forward Mode depends on the number of independent variables n.

Reverse Mode computes the gradient with respect to the number of outputs. Therefore, the computational cost of using Forward Mode depends on the number of outputs, m.

As a result, if there are more outputs than inputs, Forward mode is more efficient.
On the other hand, if there are more inputs than outputs, Reverse mode is more efficient.

### How to use `AutoDiffPy`

#### Installation
The following are the steps for installing the AutoDiffPy package and getting started. Copy the SSH url for the Github repo, then follow the below steps in your terminal.
1. `git clone <repo>`
2. `cd <repo>`
3. `pip install virtualenv` (if you don't already have it installed)
4. `python -m venv autodiff_venv`
5. `source autodiff_venv/bin/activate`
6. `pip install -r requirements.txt`

#### Running the tests
Run the command `pytest` to run all test suites. Provide an optional file parameter, e.g. `pytest tests/test_dual_number.py` to run the specified test.

To view a coverage report, run the following command `python3 -m pytest --cov=src/AutoDiffPy --cov-fail-under=90`.

#### Step-By-Step Usage
After completing the installation instructions above, follow the below steps to begin working with the `AutoDiffPy` package:

1. Import the package and `numpy` at the top of your Python file.

```
import src.AutoDiffPy as adp
import numpy as np
```

To import all functions from the package, use the following syntax, `from src.AutoDiffPy import *`, which allows you to use all functions without prepending the package name.

```
>>> from src.AutoDiffPy import *
>>> x = 2
>>> f = lambda x: sin(x)
>>> ...
```

2. Initialize variables to their desired values.

```
x = 2
y = np.sin(0.5)
z = 12352.124
```

3. Define functions using the `lambda` syntax and any relevant variables.

```
f1 = lambda x : x - 5
f2 = lambda y, z : 3 * y ** 2 - log(x)
```

4. Create a Forward Mode object and pass in a function or non-empty array of functions. The second parameter should be the number of inputs to the provided functions (dimension).

```
fwd = Forward(f1, 1)
fwd1 = Forward([f1, f2], 3)
```

5. Evaluate the functions or set of functions using the `evaluate()` method, providing the relevant variables as parameters.
```
fwd.evaluate(x)
fwd1.evaluate(x, y, z)
```

6. Compute the derivative of the function(s) using the `derivative(index, *args)` method, providing the relevant variables as parameters. For instance, `derivative(x)` will evaluate the partial derivative with respect to `x`, while `derivative(y)` will evaluate the derivative with respect to `y`. If no second parameter is provided, the function will default to differentiating with respect to the first variable of the function.

```
fwd.derivative(x)
fwd1.derivative(y)
```

7. Compute the jacobian of the function(s) using the `jacobian(*args)` method. The `args` parameter specifies the variables whose partial derivatives will be populated into the Jacobian matrix.

```
fwd.jacobian(x)
fwd1.jacobian(x, y, z)
```

### Implementation details

#### dual_number.py

The file `dual_number.py` implements the DualNumber class with support for scalar functions. For the final milestone this will expand to arbitrary dimensions.

The class has no internal dependencies and the only external dependency is Numpy.

The DualNumber class implements the following functions:
* addition and subtraction (__add__, __sub__, __radd__, __rsub__)
* multiplication and division (__mul__, __truediv__, __rmul__, __rtruediv__)
* negation (__neg__)
* comparison operators (__eq__, __leq__, __gt__, __le__, etc.)
* exponentiation (__pow__, __rpow__)

#### base_derivatives.py

The file `base_derivatives.py` implements the base derivatives for basic functions. The class depends on the `DualNumber` class and Numpy.

Each function has support for dual number, integer, or float.

The base_derivatives implements the following functions:
* Trig functions - sin, cos, tan
* Inverse trig functions - arcsin, arccos, arctan
* Hyperbolic functions - sinh, cosh, tanh
* The exponentials function, square root function, logarithmic function, power function, and sigmoid function.

These functions allow us to directly compute derivatives at each stage of the pass and store them, the evaluation of the function in the real part of the dual number, and the derivative of the function in the dual part of the dual number (in the case that we are not given a DualNumber, only an integer or float, the function is being evaluated and returned - the derivative is being discarded).

#### forward.py

The file `forward_mode.py` implements the Forward mode class  with support for single-dimensional (scalar valued) functions.

The class has a dependency on the DualNumber class (see `dual_number.py`), as the primal and tangent trace are collected in the real and dual parts of the Dual number object, respectively. The dual number class contains function overrides for elementary operations on Dual numbers, including:
* addition and subtraction (__add__, __sub__)
* multiplication and division (__mul__, __truediv__)
* negation (__neg__)
* comparison operators (__eq__, __leq__, __gt__, __le__, etc.)
* exponentiation (__pow__)

The forward mode of this AD library is supplied with an input function by the user, which may consist of any number of nested intermediate function applications. For example:
* f(a) = a
* f(b) = sin(b)
* f(c) = 1 / (tanh(c) + c)**c
* f(d) = log(c, 10) + c**2

These intermediate functions, such as `sin`, `tanh`, `log`, and exponentiation, must be both evaluated and differentiated during the AD process. The file `base_derivatives.py` computes the derivatives of these elementary functions with support for type DualNumber as well as floats and integers.



## Software Organization 

Our initial software organization plan is to store our AutoDiff classes in a source folder, our Test Suite in a test folder, an example script in an example folder, and a setup script for installing relevant packages at the root level. This provides a clear distinction between core functionality, testing, examples, and setup. 

```
team12/
├── /docs
│   └── milestone1
├── LICENSE
├── README.md
└── /src
│   └── /core
│   │   ├── base_derivatives.py (holds common basic derivatives e.g. trig)
│   │   ├── dual_number.py (holds override Dunder methods)
│   │   └── override_funcs.py (holds basic math functions for dual numbers)
│   ├── forward.py (holds forward pass class)
│   ├── reverse.py (holds reverse pass class)
│   └── /structures
│       └── graph.py (Reverse: node class)
├── /tests
│     ├── individual test files per module
│     └── test.py (runs all tests and delivers relevant output)
├── /examples
│     └── app.py
└── setup.py (runs relevant instillations)

```
At the root level of our source folder we have the base classes that form our AutoDiff implementation. Core will hold a class that includes functions that compute primitive derivatives and a dual number class with its dunder methods overriden to work as dual numbers. 

In adition to core, the source folder holds two AD method classes, forward and reverse. For reverse, we need to store the computational graph and nodes and therefore these are stored in a structures folder at the same level as reverse.

### Distribution

For the final release, we will distribute AutoDiffPy via PyPI with PEP517/518. The current state uses a simple `requirements.txt` file to manage dependencies.


## Implementation Details

This section provides a more rigorous explanation of the technical design and implementation.

### Classes

#### Base Derivatives

* Motivation: the Base Derivatives class will hold direct computations of primitive derivatives such as logarithmic, exponential, trig, and logistic functions. This will allow us to directly compute derivatives at each stage of the pass. 
* Arguments: each method will take in a function $f$ and output $f'$
* Methods: The methods will include log, exp, trig functions and there inverses, logistic.

#### Dual Number

* Motivation: the Dual Number class will construct the Dual Number objects that will be used in our Forward-Mode.
* Arguments: The `__init__` method will take in an int / float as a first argument for the real part of the Dual Number. The second argument, Dual, will be optional and default to $1$. 
* Methods: This class will have methods that override the Dunder methods of `__add__`, `__pow__`, `__mul__`, `__radd__`, `__rmul__`, `__truediv__`, `__repr__`. This operator overriding will ensure that basic mathematical operations work on Dual Numbers as well as combining them with ints and floats. Critical for the Dunder methods is that they return new Dual Numbers such that they don't return one of the inputs.

#### Override Functions

* Motivation: we need to be able to perform basic mathematical computations such as trig functions, exponentials, logarithmic, and logistic using Dual Numbers.
* Arguments: Each function will take as an argument the expression (can be a Dual Number or real number) and output the value after passing the argument through the function.
* Methods: This file will include independent methods such as: `sin(arg)`, `cos(arg)`, `exp(arg)`, `log_base(arg,base)`. These functions will operate on both Dual Numbers, capturing the Dual component as needed, and real numbers.

#### Forward

* Motivation: the Forward-Mode class runs Forward-Mode AD on a function $f$.
* Arguments: the class takes a function `f` in its `__init__` method.
* Methods:
  * compute_derivative accepts an input variable and evaluates the derivative of $f$ at the variable.
  * compute_jacobian computes the Jacobian via computing the derivative on all the input variables.   


#### Reverse 

* Motivation: the Reverse-Mode class runs Reverse-Mode AD on a function $f$. 
* Arguments: the Reverse-Mode class when initialized will accept a function $f$ and a computational graph comprising Nodes.
* Methods: the Reverse-Mode class will have $sort$, get_derivative, get_jacobian, and backward_pass methods. More on these methods in M2: extension.

#### Node

* Motivation: the Node module will hold the Node class that forms the computational graph needed for Reverse-Mode AD. 
* Node:
  * Arguments: the Node will be initialized with two arguments: a value and a local gradient. It will also hold an array of child nodes to form the graph.
  * Methods: the Node class' Dunder methods will be overriden such that they return new Nodes.

### Data Structures

The data structures used will be the computational graph via the Nodes stored in the Graph class. 

### External Libraries
* Numpy will be used for computation using vectors and matrices.
* Pytest will be used for testing our classes.




## Feedback

The feedback received from Milestone 1 was positive overall and our design does not require significant updates. In this revision, we included an additional section in `Implementation` to include the override functions for sin, cos, pow, exp, etc. as suggested in the feedback. The `Software Organization` directory was also updated to reflect this additional file.

## Future Features

#### Reverse Mode
Now that you've got most of the hard implementation work done, what kinds of things do you want to implement next?
- We plan to implement Reverse Mode AD next

How will your software change?
- Our software organization will not change because we already had accounted for reverse mode in our milestone 1 documentation. It has a `reverse.py` file in the src directory, and a `graph.py` file in the structures directory which will be the home of the node class used in reverse AD. These are currently outlined in an open pull request and will be fully implemented and tested prior to the final milestone.

What will be the primary challenges to implementing these new features?
- Reverse mode will introduce greater data structure requirements than forward mode. Dual numbers are not very useful in reverse mode since we don't apply the chain rule directly -- reverse mode computes the sensitivity of f with respect to the independent *and* intermediate variables. This means creating a new data structure we can traverse, and use basic operations on, hence the Node class.
- Another challenge is that in our design of this node class, we have to efficiently store and traverse the full computational graph! Even though there are fewer mathematical operations, there is more memory necessary for reverse mode than for foward mode, and we need to make this traversal efficient.


#### Other Extensions
We are planning to expand our project in a few further ways:
1. Adding support for multiple functions of multiple inputs
- Note that our Milestone 2 code only works for scalar functions of a single input
- Our software will change because we need to be able to take partial derivatives. In our current application, the dual part of the dual number acts as the derivative at a given step, but it does not handle taking, say, the derivative of x + y only with respect to y.
- This presents challenges because it poses the question of how we should adapt our current dual numbers class to handle partial derivatives.
- Our software will use `numpy` to handle higher-dimensional functions.

2. Writing a sample application using AutoDiffPy functionalities
- The autodifferentiation capabilities can be used for Newton's method or implementing a simple neural network.
- This brings challenges because it is fairly separate than our current work, so it will take a lot of research to efficiently create these sample applications.

3. Making our software installable via PyPI
- The current project code can be installed and set up per the "Installation" section, which involves a `requirements.txt` file. In the next iteration, it will be distributed via PyPI.

## Broader Impact
Our software provides a tool for users to efficiently compute derivatives and jacobian values. As a public package lisenced under the MIT License, this tool may be used in a number of areas ranging from scientific research, personal projects, and industry applications. While our package provides similar functionality to pre-existing AutoDiff software, releasing our own version still contributes to the public domain by providing a unique implementation and framework that others may find particularly suitable to their use case. 

As the authors of this package, there are a few consequences important to note. Firstly, while our library has undergone unit and integration testing, we cannot guarantee full correctness in all cases, especially under the conditions of large-scale usage. As with most software, this package may be subject to further iteration & fixes. There may be significant consequences if this AutoDiff library is used without user-written tests; therefore we expect any users to take responsibility for ensuring that their applications meet their expectations and that integrating this AutoDiff package does not make them susceptible to incorrect or unexpected results.

## Software Inclusivity
The aim of this package is to increase access to Autodifferentiation capabilities, for students, researchers, programmers, and others who find it applicable. Additionally, our code is released on Github to improve visibility of the internal implementation of such libraries. We welcome open-source contributions or suggestions, which can be submitted by forking this repository and creating a pull-request (this pull request can contain a simple txt file for suggestions). Pull requests will be reviewed and approved by the original authors, who may reach out to you for collaboration. As an international team, we are happy to work with individuals of any background including non-native English speakers, and will provide language accomodation if needed. For those who are visually-impaired, an audio recording of this README can be accessed in the below Google Drive link.

## License

Our software will be licensed under the MIT license, which is a highly permissive license and places few restrictions on reuse. As our software project uses standard methods of AD, we do not plan to pursue a patent. Since there are different auto differentiation packages available (such as in NumPy, PyTorch, TensorFlow, MatLab/Fortran (with packages called openAD, ADiMat), C++ (ADLOC), ADiMat, etc.), and we want other software engineers to use ours, we did not want to pick a license that would deter users. We did not add an explicit patent grant because we are not doing anything that has not been done before, so patents are not a consideration for us. Also note that since `numpy` and `pytest` have BSD licenses, we don't need to copy over their licenses.